# 앙상블
- 교재 : 210 ~ 250쪽 참고

- 보팅(Voting)
    - 서로 다른 알고리즘의 모델 여러개를 이용해서 투표로 예측 결과를 만들어 낸다.
    - 하드보팅 : 모델의 최종 예측 결과로 투표
    - 소프트보팅 : 모델의 확률결과로 투표
- 배깅(Bagging)
    - 같은 알고리즘 모델 여러개를 이용해서 투표를 진행
    - 여러개의 모델들이 학습할 때, 다른 의견이 나올 수 있도록 부트 스트랩핑이라는 추출 방법을 사용
    - 매번 모델이 학습할 때마다 랜덤하게 데이터를 추출하되, 중복된 추출을 허용한다.
    - 랜덤포레스트 : DecisionTreee의 배깅 모델
- 부스팅(Boosting)
    - 같은 알고리즘 모델 여러개를 이용해서 투표를 진행
    - 이전 모델이 잘못 학습한 결과를 다음 모델이 반영해서 학습하도록 하는 기법
    - 학습 진행이 Sequencial해서(순차적으로 일어남) 상대적으로 학습 속도가 느리다.
    - 교재 222 쪽 참고
- 스택킹(Stacking)

# Random Forest(랜덤 포레스트)

In [14]:
import pandas as pd 
X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')
X_test = pd.read_csv('data/X_test.csv')
submission = pd.read_csv('data/gender_submission.csv')

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
rf_model = RandomForestClassifier(
        n_estimators = 100,   # 깅할 모델 갯수
        max_features = 0.5,   # 각 모델이 랜덤하게 사용할 특성 수
        max_depth = 8,        # 각 모델의 최대 깊이
        min_samples_leaf = 10,# 리프 노드의 최소 샘플 수
        random_state = 72     # 난수 고정
                                 )

In [29]:
from sklearn.model_selection import cross_val_score
rf_rs = cross_val_score(rf_model, X_train, y_train, cv=5)
rf_rs

C:\Users\AI\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\AI\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\AI\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\AI\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed whe

array([0.83798883, 0.80898876, 0.85393258, 0.79213483, 0.84269663])

# 그리드 서치
- 앙상블 모델의 경우 하이퍼파리미터 수 가 엉청 많다
- 각 파라미터를 독립접으로 튜닝하느 건 불가능
- 많은 양의 파라미터를 동시에 튜닝하기 위한 함수로 그리드 서치를 지원

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
# 탐색하고 싶은 하이퍼파라미터 조합을 딕셔너리로 생성
param_grid = {
    'max_depth' : [5, 10, 15, 20],
    'n_estimators' : [100, 150, 200, 250],
    'max_features' : [0.5, 0.7],
    'min_samples_leaf' : [10, 20, 25]
}

In [32]:
model = RandomForestClassifier(random_state = 720)
# 튜닝할 모델, 사용할 파라미터, 교차검증 횟수
grid = GridSearchCV(model,param_grid, cv=3, n_jobs = -1)

In [33]:
grid.fit(X_train, y_train)

C:\Users\AI\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=720),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 20],
                         'max_features': [0.5, 0.7],
                         'min_samples_leaf': [10, 20, 25],
                         'n_estimators': [100, 150, 200, 250]})

In [35]:
# 최고의 조합 확인
# best_score : 최고 조합의 정확도 반환
# best_params : 최고 조합의 매개 변수 반환
grid.best_score_

0.8327721661054994

In [38]:
# 최고 조합으로 학습 완료된 모델
best_model = grid.best_estimator_

In [40]:
pre = best_model.predict(X_test)

In [41]:
submission['Survived'] = pre
submission.to_csv('data/rf_pre.csv', index = False)

### XG부스트
- 교재 255쪽 : XG 부스트 - 성능만 극대화 시키기 위한 모델 (부스팅 모델의 느린 속도를 많이 보완)
- 교재 258쪽 : XG 부스트 하이퍼 파라미터

### 라이트GBM
- 교재 244쪽 : 라이트GBM - XG부스트보다 속도를 조금 더 개선


# Boosting 라이브러리 설치

In [42]:
# XG부스트 설치
!pip install xgboost

In [44]:
# 라이트GBM 설치
!pip install lightgbm

In [47]:
from xgboost import XGBClassifier

In [49]:
# 하이퍼파라미터 종류(주로 쓰는 것) 
# booster = gbtree(기본값) 또는 gblinesr 또는 dart
# maxdepth
# subsmaple
# n_estimators
# learning_rate : 이전에 잘못 판단한 것을 얼마나 반영할 것인지? (값이 높을수록 많이 반영)
# gammma나 alpha 는 linear모델일 때는 사용X
xgb_model = XGBClassifier(
    booster='gbtree',    # 모델의 종류
    n_estimators = 100,  # 앙상블 모델 갯수
    learning_rate = 0.1, # 학습률
    max_depth = 4,       # 최대 깊이(일반적으로 깊게 설정X)
    gamma = 0.2          # 키우면 모델 단순, 작아지면 모델이 복잡
                         )